# Mean Encoding Tutorial
1. In this tutorial we are going to Mean encode "Embarked" column. Using four different encoding schemes.
    * K-Fold Validation
    * Leave one out 
    * Smoothing
    * Expanding mean
2. Then we are going to find corelation in between every encoding scheme and target.

# Introduction
Mean Encoding 

# Implementation


In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Reading Dataset
Train_Data = pd.read_csv("../input/pak-ec-dataset/Pakistan Largest Ecommerce Dataset.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1,2,3,7,8,9,11,12,13,14,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# About Dataset

Train_Data.shape
#Train_Data.info()
Train_Data.nunique()
Train_Data["status"].unique()

#Train_Data.head()
#Train_Data.tail(15

array(['complete', 'canceled', 'order_refunded', 'received', 'refund',
       'closed', 'fraud', 'holded', 'exchange', 'pending_paypal', 'paid',
       '\\N', 'cod', 'pending', nan, 'processing', 'payment_review'],
      dtype=object)

In [3]:
Train_Data.drop(Train_Data.iloc[:, 9:26], axis=1, inplace=True)

In [4]:
Train_Data.head()
Train_Data.shape

(1048575, 9)

# Mean Encoding with K-Fold Validation
***Steps***
1. **Import K-Fold with K=5 and shuffle = False**
2. **Split Train data in to Tr_d and Val_d**
3. **Iterate through K folds (If K=x This iteration will repeat x times)**
    * Train and Validation -(Assign)- train and validation index
    * Find Target mean for Train Folds groupedby(Categorical feature under operation)
    * New Feature in Validation Fold -(Assign)- Validation fold (Categorical feature under operation) .map(Target mean of Train folds)
    * Dataset(Index val) -(Assign)- Validation fold
4. **Fill null values with global mean in new feature of Dataset**